# LangChain Tool calling capabilities with Unity Catalog

## Prerequisites

**API Key**
To run this tutorial, you will need an OpenAI API key. 

Once you have acquired your key, set it to the environment variable `OPENAI_API_KEY`.

Below, we validate that this key is set properly in your environment.

**Packages**

To interface with both UnityCatalog and LangChain, you will need to install the following packages:

```shell
pip install langchain_openai unitycatalog-langchain
```

In [1]:
import os

assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

## Configuration and Client setup

In order to connect to your Unity Catalog server, you'll need an instance of the `ApiClient` from the `unitycatalog-client` package. 

> Note: If you don't already have a Catalog and a Schema created, be sure to create them before running this notebook and adjust the `CATALOG` and `SCHEMA` variables below to suit.

In [ ]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit
from unitycatalog.client import ApiClient, Configuration

In [ ]:
config = Configuration(host="http://localhost:8080/api/2.1/unity-catalog")

api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "AICatalog"
SCHEMA = "AISchema"

## Define functions and register them to Unity Catalog

In this next section, we'll be defining two Python functions and creating them within Unity Catalog so that they can be retrieved and used as tools within our LangChain Agent. 

There are a few things to keep in mind when creating functions for use with the `create_python_function` API:

- Ensure that your have properly defined types for all arguments and for the return of the function.
- Ensure that you have a Google-style docstring defined that includes descriptions for the function, each argument, and the return of the function. This is critical, as these are used to populate the metadata associated with the function within Unity Catalog, providing contextual data for an LLM to understand when and how to call the tool associated with this function.
- If there are packages being called that are not part of core Python, ensure that the import statements are locally scoped (defined within the function body).

In [4]:
def calculate_required_laser_power(
    material: str, thickness: float, cutting_speed: float, laser_efficiency: float = 0.8
) -> float:
    """
    Calculates the minimum laser power required to cut through a specific material.

    The required laser power is determined based on the material's thermal properties,
    thickness, and the desired cutting speed. The formula used is:

        P = (k * thickness * cutting_speed) / (η * A)

    where:
        - P is the required laser power (W).
        - k is the thermal conductivity of the material (W/m·K).
        - thickness is the material thickness (m).
        - cutting_speed is the speed at which the laser moves (m/s).
        - η is the laser efficiency (dimensionless).
        - A is the absorption coefficient of the material (dimensionless, between 0 and 1).

    Args:
        material (str): Type of the material (e.g., 'steel', 'aluminum', 'copper').
        thickness (float): Thickness of the material in meters (m).
        cutting_speed (float): Desired cutting speed in meters per second (m/s).
        laser_efficiency (float, optional): Efficiency of the laser system (default is 0.8).

    Returns:
        Required laser power in watts (W).
    """

    material_properties = {
        "steel": {"thermal_conductivity": 50, "absorption_coefficient": 0.5},
        "aluminum": {"thermal_conductivity": 205, "absorption_coefficient": 0.6},
        "copper": {"thermal_conductivity": 385, "absorption_coefficient": 0.55},
        "titanium": {"thermal_conductivity": 22, "absorption_coefficient": 0.45},
        "brass": {"thermal_conductivity": 110, "absorption_coefficient": 0.5},
        "polycarbonate": {"thermal_conductivity": 0.2, "absorption_coefficient": 0.3},
        "acrylic": {"thermal_conductivity": 0.2, "absorption_coefficient": 0.35},
    }

    if material.lower() not in material_properties:
        raise ValueError(
            f"Unsupported material: {material}. Supported materials are: {list(material_properties.keys())}"
        )

    if thickness <= 0:
        raise ValueError("Thickness must be a positive value.")
    if cutting_speed <= 0:
        raise ValueError("Cutting speed must be a positive value.")
    if not (0 < laser_efficiency <= 1):
        raise ValueError("Laser efficiency must be between 0 (exclusive) and 1 (inclusive).")

    properties = material_properties[material.lower()]
    k = properties["thermal_conductivity"]  # W/m·K
    A = properties["absorption_coefficient"]  # Dimensionless

    power = (k * thickness * cutting_speed) / (laser_efficiency * A)

    return power


def calculate_cutting_speed(
    material: str, thickness: float, laser_power: float, laser_efficiency: float = 0.8
) -> float:
    """
    Calculates the maximum cutting speed achievable with a given laser power.

    The cutting speed is determined based on the material's thermal properties,
    thickness, and the available laser power. The formula used is:

        cutting_speed = (P * η * A) / (k * thickness)

    where:
        - P is the laser power (W).
        - η is the laser efficiency (dimensionless).
        - A is the absorption coefficient of the material (dimensionless, between 0 and 1).
        - k is the thermal conductivity of the material (W/m·K).
        - thickness is the material thickness (m).

    Args:
        material (str): Type of the material (e.g., 'steel', 'aluminum', 'copper').
        thickness (float): Thickness of the material in meters (m).
        laser_power (float): Available laser power in watts (W).
        laser_efficiency (float, optional): Efficiency of the laser system (default is 0.8).

    Returns:
        Maximum cutting speed in meters per second (m/s).
    """
    material_properties = {
        "steel": {"thermal_conductivity": 50, "absorption_coefficient": 0.5},
        "aluminum": {"thermal_conductivity": 205, "absorption_coefficient": 0.6},
        "copper": {"thermal_conductivity": 385, "absorption_coefficient": 0.55},
        "titanium": {"thermal_conductivity": 22, "absorption_coefficient": 0.45},
        "brass": {"thermal_conductivity": 110, "absorption_coefficient": 0.5},
        "polycarbonate": {"thermal_conductivity": 0.2, "absorption_coefficient": 0.3},
        "acrylic": {"thermal_conductivity": 0.2, "absorption_coefficient": 0.35},
    }

    if material.lower() not in material_properties:
        raise ValueError(
            f"Unsupported material: {material}. Supported materials are: {list(material_properties.keys())}"
        )

    if thickness <= 0:
        raise ValueError("Thickness must be a positive value.")
    if laser_power <= 0:
        raise ValueError("Laser power must be a positive value.")
    if not (0 < laser_efficiency <= 1):
        raise ValueError("Laser efficiency must be between 0 (exclusive) and 1 (inclusive).")

    properties = material_properties[material.lower()]
    k = properties["thermal_conductivity"]
    A = properties["absorption_coefficient"]

    speed = (laser_power * laser_efficiency * A) / (k * thickness)

    return speed

In [5]:
client.create_python_function(
    func=calculate_required_laser_power, catalog=CATALOG, schema=SCHEMA, replace=True
)

FunctionInfo(name='calculate_required_laser_power', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='material', type_text='STRING', type_json='{"name": "material", "type": "string", "nullable": false, "metadata": {"comment": "Type of the material (e.g., \\"steel\\", \\"aluminum\\", \\"copper\\")."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='Type of the material (e.g., "steel", "aluminum", "copper").'), FunctionParameterInfo(name='thickness', type_text='DOUBLE', type_json='{"name": "thickness", "type": "double", "nullable": false, "metadata": {"comment": "Thickness of the material in meters (m)."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None,

In [6]:
client.create_python_function(
    func=calculate_cutting_speed, catalog=CATALOG, schema=SCHEMA, replace=True
)

FunctionInfo(name='calculate_cutting_speed', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='material', type_text='STRING', type_json='{"name": "material", "type": "string", "nullable": false, "metadata": {"comment": "Type of the material (e.g., \\"steel\\", \\"aluminum\\", \\"copper\\")."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='Type of the material (e.g., "steel", "aluminum", "copper").'), FunctionParameterInfo(name='thickness', type_text='DOUBLE', type_json='{"name": "thickness", "type": "double", "nullable": false, "metadata": {"comment": "Thickness of the material in meters (m)."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parame

## Create a Toolkit instance of the functions

Now that the functions have been created within Unity Catalog, we can use the `unitycatalog-langchain` package to create a toolkit instance that our Agent will 'understand' as valid tools to use within its APIs. 

In [7]:
laser_power_func = f"{CATALOG}.{SCHEMA}.calculate_required_laser_power"
cutting_speed_func = f"{CATALOG}.{SCHEMA}.calculate_cutting_speed"

toolkit = UCFunctionToolkit(function_names=[laser_power_func, cutting_speed_func], client=client)

tools = toolkit.tools

In [8]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant for industrial tasks. When asked a question, determine if there are appropriate"
            "tools that can be employed to answer the question with factual context.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke(
    {
        "input": "I want to make a sunroof in a shipping container. My friend has a 5 kw laser. The shipping container is made "
        "of 0.25inch thick mild steel and is a standard 40 foot ISO container. I want to see if I can get this done in an afternoon."
    }
)



> Entering new AgentExecutor chain...

Invoking: `AICatalog__AISchema__calculate_cutting_speed` with `{'material': 'steel', 'thickness': 0.00635, 'laser_power': 5000}`


{"error": "calculate_cutting_speed() missing 1 required positional argument: 'laser_efficiency'"}
Invoking: `AICatalog__AISchema__calculate_required_laser_power` with `{'material': 'steel', 'thickness': 0.00635, 'cutting_speed': 1}`


{"error": "calculate_required_laser_power() missing 1 required positional argument: 'laser_efficiency'"}
Invoking: `AICatalog__AISchema__calculate_cutting_speed` with `{'material': 'steel', 'thickness': 0.00635, 'laser_power': 5000, 'laser_efficiency': 0.8}`


{"format": "SCALAR", "value": "6299.212598425197"}
Invoking: `AICatalog__AISchema__calculate_required_laser_power` with `{'material': 'steel', 'thickness': 0.00635, 'cutting_speed': 1, 'laser_efficiency': 0.8}`


{"format": "SCALAR", "value": "0.79375"}Here are the results for your sunroof project in the shipping container:

1. **

{'input': 'I want to make a sunroof in a shipping container. My friend has a 5 kw laser. The shipping container is made of 0.25inch thick mild steel and is a standard 40 foot ISO container. I want to see if I can get this done in an afternoon.',
 'output': 'Here are the results for your sunroof project in the shipping container:\n\n1. **Maximum Cutting Speed**: With a 5 kW laser and assuming a laser efficiency of 80%, the maximum cutting speed you can achieve on 0.25 inch thick mild steel is approximately **6.30 meters per second**.\n\n2. **Required Laser Power for a Cutting Speed of 1 m/s**: To achieve a cutting speed of 1 meter per second, you would require approximately **0.79 kW (or 793.75 W)** of laser power.\n\n### Conclusion\nSince your 5 kW laser exceeds the required power to achieve a cutting speed of 1 m/s, you should be able to cut the sunroof in an afternoon, depending on the actual cutting path length and any additional setup time needed.'}